In [ ]:
import sys
from pathlib import Path

# Add pub_worm directory to the Python path
sys.path.insert(0, str(Path.cwd().parent))

# Test where we are loading the pub_worm objects from
import inspect
from pub_worm.wormbase.wormbase_api import WormbaseAPI

module = inspect.getmodule(WormbaseAPI)
if hasattr(module, "__file__"):
    print("WormbaseAPI imported from:", module.__file__)
else:
    print("Could not determine the file path.")

In [ ]:
import pandas as pd
wS296_gene_ids_df = pd.read_csv("./wormbase_data/c_elegans.PRJNA13758.WS296.geneIDs.csv")
excel_file = "./wormbase_data/1-s2.0-mmc6.xlsx"
sheets_dict = pd.read_excel(excel_file, sheet_name=None)


In [ ]:
gene_ids_df = pd.read_csv("./wormbase_data/c_elegans.PRJNA13758.WS296.geneIDs.csv").fillna('')

gene_ids_dict = {}
for _, row in gene_ids_df.iterrows():
    for key in ['Wormbase_Id', 'Gene_name', 'Sequence_id']:
        id_val = str(row[key]).upper()
        gene_ids_dict[id_val] = row.to_dict()
            

In [ ]:
from pub_worm.wormbase.wormbase_util import _lookup_wormbase_id

def lookup_wormbase_id(sequence_id, gene_ids_dict):
    results_dict = _lookup_wormbase_id(sequence_id, gene_ids_dict)
    if results_dict:
        return results_dict['Wormbase_Id']
    return None
    

In [ ]:
from pub_worm.wormbase.wormbase_util import map_wormbase_ids,_lookup_wormbase_id
 
results_dict = {}

for sheet_name, df in sheets_dict.items():
    if 'genename' in df.columns:
        df['Wormbase_Id'] = df['genename'].apply(lambda g: lookup_wormbase_id(g, gene_ids_dict))
        results_dict[sheet_name] = df
    


In [ ]:
# from pub_worm.wormbase.wormbase_util import map_wormbase_ids,_lookup_wormbase_id
 
# results_dict = {}

# for sheet_name, df in sheets_dict.items():
#     # Sanitize the sheet name to avoid issues in filenames
#     safe_name = sheet_name.replace(" ", "_").replace("/", "_")
#     csv_filename = f"./wormbase_data/{safe_name}.csv"
    
#     df.to_csv(csv_filename, index=False)
#     try:
#         df = map_wormbase_ids(csv_filename, column_name='genename', gene_ids_df=wS296_gene_ids_df)
#         results_dict[sheet_name] = df
#         print(f"Saved: {csv_filename}")
#     except Exception as e:
#         continue


In [ ]:
with pd.ExcelWriter("results2.xlsx") as writer:
    for sheet_name, df in results_dict.items():
        # Write to the Excel file with the sheet named after the key
        if 'Wormbase_Id' in df.columns:
            # Create a new column order with Wormbase_Id first
            cols = ['Wormbase_Id'] + [col for col in df.columns if col != 'Wormbase_Id']
            df = df[cols]
        # Write the DataFrame to the specified sheet    
        df.to_excel(writer, sheet_name=sheet_name, index=False)